# 앙상블
- 단일 결정트리(Decision Tree)의 단점을 극복하기 위해 여러 머신러닝 모델을 연결하여 더 강력한 모델을 만드는 방법
- 주어진 자료로부터 여러 개의 예측 모형을 만든 후 예측 모형을 조합하여 하나의 예측 모형을 생성하는 과정
- 대표적인 기법: 배깅, 부스팅, 랜덤 포레스트

## 배깅
- 주어진 자료를 모집단으로 간주하여, 주어진 자료에서 여러 개의 부트 스트랩 자료를 생성 -> 각각의 자료에 예측 모형을 만듦 -> 결합하여 최종 예측 모델 생성

- 모델 하나를 정해놓고 학습 데이터를 바꿔가며 예측도를 만들어가는 방식

- 장점
    - 분산을 줄이고 과적합 문제를 완화
    - 안정적인 성능 보장
    - 비선형성 데이터와 노이즈 데이터에 대한 문제 완화
- 매개 변수
    - <span style="color:#ffd33d">**estimator**</span>
        </br>: 기본 추정기 (모델 선택)
        - 기본값: None
    - <span style="color:#ffd33d">**n_estimators**</span>
        </br>: 생성할 추정기(모델)의 개수
        - 기본값: 10
    - <span style="color:#ffd33d">**max_samples**</span>
        </br>: 모델에 사용할 데이터의 비율
        - 과적합 문제 완화 가능
        - 기본값: 1.0
            - 1.0 : 모든 데이터를 사용
    - <span style="color:#ffd33d">**max_features**</span>
        </br>: 모델에 사용할 데이터 컬럼의 비율
        - 중복된 데이터가 많은 경우 사용
        - 과적합 문제 완화 가능
        - 기본값: 1.0
    - <span style="color:#ffd33d">**oob_score**</span>
        </br>: oob(out-of-bag) 데이터로 일반화 성능을 평가할 것인가?
        - 기본값: False
    - <span style="color:#ffd33d">**bootstrap**</span>
        </br>: 샘플링 데이터 이용 시 중복 데이터 허용 여부
        - 기본값: True
            - True : 중복 허용
            - False : 중복 미허용
    - <span style="color:#ffd33d">**bootstrap_features**</span>
        </br>: 샘플링 데이터 이용 시 중복 컬럼 허용 여부
        - 기본값: False
            - True : 중복 허용
            - False : 중복 미허용
    - <span style="color:#ffd33d">**warm_start**</span>
        </br>: 이전 학습 결과에 이어서 추가 학습할 것인가?
        - 기본값: False
    - <span style="color:#ffd33d">**n_jobs**</span>
        </br>: 해당 작업 수행 시 사용할 CPU의 병렬처리 개수
        - 기본값: None
            - -1 : 전체 CPU 사용

- 속성 (변수)
    - <span style="color:#ffd33d">**estimators_**</span>
        </br>: 학습된 모델의 리스트
    - <span style="color:#ffd33d">**estimators_samples_**</span>
        </br>: 각 모델이 학습한 샘플의 인덱스 (= 행의 위치)
    - <span style="color:#ffd33d">**estimators_fetures_**</span>
        </br>: 각 모델이 학습한 샘플의 컬럼의 인덱스 (= 열의 위치)
    - <span style="color:#ffd33d">**oob_score_**</span>
        </br>: oob 데이터 기반 정확도
    - <span style="color:#ffd33d">**oob_decision_function_**</span>
        </br>: oob 데이터의 클래스별 예측 확률
    - <span style="color:#ffd33d">**classes_**</span>
        </br>: 분류 클래스의 레이블
    - <span style="color:#ffd33d">**n_classes_**</span>
        </br>: 클래스의 개수

- 메소드 (함수)
    - <span style="color:#ffd33d">**fit(X, y)**</span>
        </br>: 모델 학습
    - <span style="color:#ffd33d">**predict(X)**</span>
        </br>: 모델의 예측
    - <span style="color:#ffd33d">**predict_proba(X)**</span>
        </br>: 클래스별 예측 확률
        - 추정기(모델)이 predict_proba( )를 지원하는 경우에만 사용 가능
    - <span style="color:#ffd33d">**score(X, y)**</span>
        </br>: 정확도 평가
    - <span style="color:#ffd33d">**fit_predict(X, y)**</span>
        </br>: 학습 데이터를 이용한 예측
        - 학습 후 바로 예측 수행

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
breast = pd.read_csv('../data/breast-cancer.csv')

In [ ]:
breast.head()

In [ ]:
# diagcosis 데이터의 개수를 확인 
breast['diagnosis'].value_counts()

In [ ]:
sns.countplot( data=breast, x = 'diagnosis' )

In [ ]:
breast.info()

In [ ]:
# 특정 컬럼을 선택하여 학습, 예측
# 독립 변수 area_mean, texture_mean 
# 산점도 그래프 
sns.relplot(
    data = breast, 
    x = 'area_mean', 
    y = 'texture_mean', 
    hue = 'diagnosis'
)

In [ ]:
# 진단결과 diagnosis의 데이터를 M은 1로 B는 0으로 변환 
breast['class'] = breast['diagnosis'].map(
    {
        'M' : 1, 
        'B' : 0
    }
)

In [ ]:
# 독립 변수, 종속 변수 
x = breast[[ 'area_mean', 'texture_mean' ]].values
y = breast['class'].values

In [ ]:
# train, test로 데이터를 분할
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size= 0.3, 
    random_state=42, 
    stratify= y
)

In [ ]:
# 배깅을 이용하여 분류기를 생성하고 예측 
clf = BaggingClassifier(estimator=DecisionTreeClassifier())

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
# 예측 
pred = clf.predict(X_test)

In [ ]:
# class.score를 이용하여 정확도를 확인 
clf.score(X_test, Y_test)

In [ ]:
# 혼동행렬을 표시 
pd.DataFrame(
    confusion_matrix(Y_test, pred), 
    index = ['actual[0]', 'actual[1]'], 
    columns = ['pred[0]', 'pred[1]']
)

In [ ]:
tree = DecisionTreeClassifier()

In [ ]:
tree.fit(X_train, Y_train)

In [ ]:
pred_tree = tree.predict(X_test)

In [ ]:
# Decisiontree 정확도 
tree.score(X_test, Y_test)

In [1]:
confusion_matrix(Y_test, pred_tree)

NameError: name 'confusion_matrix' is not defined

In [ ]:
# oob score 사용 clf_oob = BaggingClassifier(estimator= DecisionTreeClassifier(max_depth=4), oob_score=True)

In [ ]:
clf_oob.fit(X_train, Y_train)

In [ ]:
clf_oob.oob_score_

In [ ]:
clf_oob.score(X_test, Y_test)

In [ ]:
# 배깅 방식으로 회귀 분석 
# 자동차의 정보가 있는 데이터셋 로드 
car = pd.read_csv(
    "../data/CarPrice_Assignment.csv"
)

In [ ]:
car.head()

In [ ]:
car.info()

In [ ]:
# car 데이터프레임에서 dtype이 숫자형은 컬럼들만 필터 
# 선택(select) + 데이터의 타입(dtypes)
# 내가 원하는 dtypes 컬럼만 선택
car_num = car.select_dtypes('number')

In [ ]:
# 독립 변수, 종속 변수 
x = car_num.drop(['car_ID', 'symboling', 'price'], axis=1).values
y = car_num['price'].values

In [ ]:
# 독립 변수, 종속 변수 
x = car_num.drop(['car_ID', 'symboling', 'price'], axis=1).values
y = car_num['price'].values

In [ ]:
X_train, X_test, Y_train, Y_test =train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [ ]:
reg = BaggingRegressor(estimator=DecisionTreeRegressor(), oob_score=True)reg = BaggingRegressor(estimator=DecisionTreeRegressor(), oob_score=True)

In [ ]:
reg.fit(X_train, Y_train)

In [ ]:
# oob_score 확인
reg.oob_score_

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
pred_reg = reg.predict(X_test)

In [ ]:
r2_score(Y_test, pred_reg)